In [1]:
import os
from config import opt
from main import train
from main import test
import ipywidgets as widgets
from IPython.display import display
from IPython.display import clear_output
from ipywidgets import (interact, interactive, fixed)
from utils import file_selector
from evaluation import load_result
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from pylab import *
from datasets import Test_Dataset

dataset_name = widgets.Dropdown(
                        options=['Market1501', 'DukeMTMC'],
                        value='Market1501',
                        description='Dataset:',
                        disabled=False,
                    )
model = widgets.Dropdown(
                    options=['ResNet50', 'DenseNet121'],
                    value='ResNet50',
                    description='Model:',
                    disabled=False,
                )

re_ranking = widgets.Checkbox(
                value=True,
                description='Re_Ranking:',
                disabled=False
            )


train_button = widgets.Button(description="Train")
test_button = widgets.Button(description='Test')
items = [train_button,test_button]
button = widgets.Box(items)

opt.dataset_name = dataset_name.value
opt.model = model.value
opt.re_ranking = re_ranking.value
#display(dataset_name,model,re_ranking,button)
out = widgets.Output()
def click_train(b):
    with out:
        clear_output()
        train()
def click_test(b):
    with out:
        clear_output()
        test()
train_button.on_click(click_train)
test_button.on_click(click_test)
widgets.VBox([dataset_name,model,re_ranking,button, out])

VBox(children=(Dropdown(description='Dataset:', options=('Market1501', 'DukeMTMC'), value='Market1501'), Dropdown(description='Model:', options=('ResNet50', 'DenseNet121'), value='ResNet50'), Checkbox(value=True, description='Re_Ranking:'), Box(children=(Button(description='Train', style=ButtonStyle()), Button(description='Test', style=ButtonStyle()))), Output()))

In [2]:
def input_result_demo(fname,R,size):
    result,CMC,mAP = load_result()
    ranking = result['ranking']
    query_imgs_path = result['query_imgs_path']
    gallery_imgs_path = result['gallery_imgs_path']

    img_path = os.path.join(opt.data_dir,opt.dataset_name,'query',fname)
    
    query_index = np.where(query_imgs_path==img_path)

    index = query_index[0][0]

    query_label = query_imgs_path[index].split('/')[-1].split('_')[0]

    fig, img = plt.subplots(1,R+1,figsize=(size,size))
    for i in range(0,R+1):
        if i == 0:
            img[i].imshow(mpimg.imread(query_imgs_path[index]))
            img[i].set_title('Query Image \n ID:'+str(int(query_label)))
        else:    
            img[i].imshow(mpimg.imread(gallery_imgs_path[int(ranking[index][i-1])]))
            gallery_label = gallery_imgs_path[int(ranking[index][i-1])].split('/')[-1].split('_')[0]
            if i == 1:
                img[i].set_title('Gallery Images \n ID:'+str(int(gallery_label)))
            else:
                img[i].set_title('ID:'+str(int(gallery_label)))

            if(gallery_label== query_label):
                autoAxis = img[i].axis()
                rec = Rectangle((autoAxis[0]-0.7,autoAxis[2]-0.2),(autoAxis[1]-autoAxis[0])+1,(autoAxis[3]-autoAxis[2])+0.4,fill=False,lw=2,color='red')
                rec = img[i].add_patch(rec)
                rec.set_clip_on(False)
        img[i].axis('off')

    plt.show()
    
file = widgets.Button(description='Open Query Image')
out = widgets.Output()

def click(b): 
    file_path = file_selector(opt.data_dir).decode("utf-8")
    fname = str(file_path).split('/')[-1]
    file.description = fname
    with out:
        #clear_output()
        interact(input_result_demo, 
         fname = fname,
         R=widgets.IntSlider(value=5,min=0,max=20), 
         size = widgets.IntSlider(value=20,min=0,max=40)
        )

file.on_click(click)
widgets.VBox([file, out])

VBox(children=(Button(description='Open Query Image', style=ButtonStyle()), Output()))

In [3]:
def select_result_demo(R,index,size):
    result,CMC,mAP = load_result()
    ranking = result['ranking']
    query_imgs_path = result['query_imgs_path']
    gallery_imgs_path = result['gallery_imgs_path']


    query_label = query_imgs_path[index].split('/')[-1].split('_')[0]

    fig, img = plt.subplots(1,R+1,figsize=(size,size))
    for i in range(0,R+1):
        if i == 0:
            img[i].imshow(mpimg.imread(query_imgs_path[index]))
            img[i].set_title('Query Image \n ID:'+str(int(query_label)))
        else:    
            img[i].imshow(mpimg.imread(gallery_imgs_path[int(ranking[index][i-1])]))
            gallery_label = gallery_imgs_path[int(ranking[index][i-1])].split('/')[-1].split('_')[0]
            if i == 1:
                img[i].set_title('Gallery Images \n ID:'+str(int(gallery_label)))
            else:
                img[i].set_title('ID:'+str(int(gallery_label)))

            if(gallery_label== query_label):
                autoAxis = img[i].axis()
                rec = Rectangle((autoAxis[0]-0.7,autoAxis[2]-0.2),(autoAxis[1]-autoAxis[0])+1,(autoAxis[3]-autoAxis[2])+0.4,fill=False,lw=2,color='red')
                rec = img[i].add_patch(rec)
                rec.set_clip_on(False)
        img[i].axis('off')

    plt.show()
    
interact(select_result_demo, 
         R=widgets.IntSlider(value=5,min=0,max=20), 
         index = widgets.IntSlider(value=0,min=0,max=len(Test_Dataset('query'))-1) ,
         size = widgets.IntSlider(value=20,min=0,max=40)
        )

interactive(children=(IntSlider(value=5, description='R', max=20), IntSlider(value=0, description='index', max=3367), IntSlider(value=20, description='size', max=40), Output()), _dom_classes=('widget-interact',))

<function __main__.select_result_demo>